## TRACER 17 JUNE 2022

### Lightning for all RHIs

[Lightning Data for June 17th](http://pogo.tosm.ttu.edu/data/TRACER/)

In [1]:
import matplotlib.pyplot as plt
import pyart
import numpy as np
import cartopy.crs as ccrs
import pandas as pd
import xarray as xr
from datetime import datetime, date, time, timedelta
import pyart.graph.cm as pcm
import warnings
warnings.filterwarnings("ignore")

from lmatools.coordinateSystems import RadarCoordinateSystem, GeographicSystem, TangentPlaneCartesianSystem, MapProjection


## You are using the Python ARM Radar Toolkit (Py-ART), an open source
## library for working with weather radar data. Py-ART is partly
## supported by the U.S. Department of Energy as part of the Atmospheric
## Radiation Measurement (ARM) Climate Research Facility, an Office of
## Science user facility.
##
## If you use this software to prepare a publication, please cite:
##
##     JJ Helmus and SM Collis, JORS 2016, doi: 10.5334/jors.119



<frozen importlib._bootstrap>:283: DeprecationWarning: the load_module() method is deprecated and slated for removal in Python 3.12; use exec_module() instead


### [LMAinterceptRHI](https://github.com/jcssouza/LMAinterceptRHI)

In [2]:
import sys
sys.path.append("./LMAinterceptRHI/")
# Adapted functions from LMAinterceptRHI
from radarlma2local import geo_to_tps
from ortho_proj import rot_mat_lma, close_sources, closest_pt_radarloc
from radar_processing import r_z_centers_edges_mesh
from interp_funcs import centers_to_edges_1d, coords_2d

In [3]:
from shapely.geometry import Polygon, Point, MultiPoint

class RadarLMASubsetter:
    def __init__(self, radar, lma, sweep_id=0):
        self.radar = radar
        self.lma = lma
        
        # -- Get radar time
        d = date(int(radar.time['units'][14:18]),int(radar.time['units'][19:21]),int(radar.time['units'][22:24]))
        t = time(int(radar.time['units'][25:27]),int(radar.time['units'][28:30]),int(radar.time['units'][31:33]))
        self.radar_time = datetime.combine(d,t)
        
        print('Radar scan type: ' + radar.scan_type)
        print('Sweep: ', sweep_id, 'of', radar.nsweeps)
        print(self.radar_time)
        
        # -- Initialize coordinate systems
        ctrlat, ctrlon, ctralt = radar.latitude['data'][0], radar.longitude['data'][0], radar.altitude['data'][0]
        self.geo = GeographicSystem()
        self.tps = TangentPlaneCartesianSystem(ctrlat, ctrlon, ctralt)
        self.rcs = RadarCoordinateSystem(ctrlat, ctrlon, ctralt)

    
        # -- Radar scan geometry: centers and edges of each gate
        RZ_centers, RZ_edges = r_z_centers_edges_mesh(radar, sweep_id)
        self.r_c = RZ_centers[:,:,0]
        self.z_c = RZ_centers[:,:,1]
        self.r_e = RZ_edges[:,:,0]
        self.z_e = RZ_edges[:,:,1]

        # Radar shape
        # -- To filter VHF sources out of the radar scan domain
        # --- Create the polygon: all ranges, lowest and highest elevation
        self.scan_poly = Polygon(np.append(
                np.append(RZ_edges[-1, :-1], RZ_edges[0, :-1], axis = 0), 
                RZ_edges[:, - 1], axis = 0))
        # scan_poly
        
    
    def lightning_within_time(self, seconds, max_alt=18e3, max_chi2=1.0, min_station=6):
        lma_data = self.lma
        radar_time = self.radar_time
        
        # Filter time and altitude
        sources = (lma_data.event_altitude < max_alt) & (abs( (np.datetime64(radar_time) - lma_data.event_time)/ np.timedelta64(1, 's') ) < seconds) 
        subset = lma_data[{'number_of_events': sources}]

        flashes_ids = np.unique(subset.event_parent_flash_id).astype(int)
        print(f'{len(flashes_ids)} flashes in {2*seconds} second interval near scan.')

        
        # Select events info for individual flash id ('number_of_events' dimension)
        # nid = 64
        # one_id = lma_data.event_parent_flash_id == flashes_ids[nid]
        # flash_events = lma_data[{'number_of_events': one_id}]

        # Filter data
        filter_events = (subset.event_chi2 < max_chi2) & (subset.event_stations > min_station)
        filter_events
        filtered = subset[{'number_of_events':filter_events}]

        # Only one flash events info in the dataset
        all_dims = dict(filtered.dims)
        all_dims.pop('number_of_events') #all dims will no longer have 'number of flashes’
        return filtered.drop_dims(all_dims.keys())
    
    def plot_lightning_near_scan_time(self, seconds,  **kwargs):
        radar = self.radar
        
        events_near = self.lightning_within_time(seconds, **kwargs)
        # Quick look at RHI relative to the selected lightning before coordinate transformations
        fig, ax = plt.subplots(1,1)
        flash_time = np.sort(events_near.event_time)
        loc_lma = ax.scatter(events_near.event_longitude, events_near.event_latitude,
                              marker = '.', s = 9, c = (flash_time - flash_time[0]), cmap = 'viridis',alpha=0.9)
        loc_radar = ax.scatter(radar.longitude['data'],radar.latitude['data'],
                                color = 'brown', marker = 's', s = 60)
        loc_rhi = ax.plot([radar.longitude['data'], radar.gate_longitude['data'][0][-1]],
                           [radar.latitude['data'], radar.gate_latitude['data'][0][-1]],
                           color = 'black')
        return ax
        # plt.axis((-95.5, -95.3, 29.65, 29.85))

    def find_lightning_distances_near_scan_time(self, seconds, distance, **kwargs):
        events_near = self.lightning_within_time(seconds, **kwargs)
        
        radar = self.radar

        # --- Orthogonal projection/ Matrix Rotation of the LF sources
        Xlma,Ylma,Zlma = geo_to_tps(events_near, radar)   # Filter does not have altitude
        XYZlma = np.column_stack((Xlma, Ylma, Zlma))
        lma_file_ortho = rot_mat_lma(radar, XYZlma, -1)  # -1 for counterclockwise

        # 1st STEP
        # -- Find and store sources within a certain ds distance
        ds = distance # m threshold
        r_cls, z_cls, y_min = close_sources(self.r_c, self.z_c, lma_file_ortho, ds)
        # -- Selected LMA sources inside the radar scan
        lma_shp = MultiPoint(tuple(np.vstack((r_cls, z_cls)).transpose()))
        R_cls = [] 
        Z_cls = []
        Y_min = []
        for i in np.arange(len(r_cls)):
            if self.scan_poly.contains(lma_shp[i]) == True:
                R_cls.append(np.asarray(lma_shp[i].coords[0])[0])
                Z_cls.append(np.asarray(lma_shp[i].coords[0])[1])
                Y_min.append(y_min[i])
        R_cls = np.asarray(R_cls)
        Z_cls = np.asarray(Z_cls)
        Y_min = np.asarray(Y_min)
        
        return R_cls, Z_cls, Y_min
    
    def radar_gate_at_lightning_source(self, R_cls, Zcls, Y_min):
        
        radar = self.radar
        
        # ASSUME Y = 0 is the location of the scan plane
        int_point1 = [np.array([R_cls[i], 0 , Z_cls[i]]) for i in np.arange(R_cls.size)]

        # 2o STEP
        # Transform from Rotationed Coordinate system Tangent Plane to Tangent Plan (rotates clockwise = 1)
        int_point2 = rot_mat_lma(radar, int_point1, 1)

        # 3o STEP
        # Tangent Plan to ECEF
        int_point3 = [self.tps.fromLocal(int_point2[i,:][:,None]) for i in np.arange(R_cls.size)]

        # 4o STEP
        # Transform from ECEF to Radar Coordinate System
        int_point4 = [self.rcs.fromECEF(int_point3[i][0],int_point3[i][1],int_point3[i][2]) for i in np.arange(R_cls.size)]

        # 5o STEP
        # Find closest point to RHI - Elevation and Range
        cls_r_idx = [closest_pt_radarloc(radar, int_point4[i])[0]  for i in np.arange(R_cls.size)]    # Radius index
        cls_az_idx = [closest_pt_radarloc(radar, int_point4[i])[1]  for i in np.arange(R_cls.size)]   # Azimuth index
        cls_elev_idx = [closest_pt_radarloc(radar, int_point4[i])[2]  for i in np.arange(R_cls.size)] # Elevation index
        
        return cls_r_idx, cls_az_idx, cls_elev_idx

In [4]:
class OverlayPlotter:
    
    def __init__(self, subsetter, seconds, distance):
        self.subsetter = subsetter
        self.radar = subsetter.radar
        self.r_e, self.z_e = subsetter.r_e, subsetter.z_e
        self.R_cls, self.Z_cls, self.Y_min = subsetter.find_lightning_distances_near_scan_time(seconds, distance)
        
    def plot(self, field='reflectivity', 
             cbar_range={'reflectivity':(0,60)},
             units={'reflectivity':'dBZ'},
             mask_field = 'reflectivity',
             mask_field_thresh=-10, distance_max = 1000):
        
        radar = self.radar
        R_cls, Z_cls, Y_min = self.R_cls, self.Z_cls, self.Y_min
        r_e, z_e = self.r_e, self.z_e
        
        # -- For plots below
        # -- Selecting coordinates and values for images below
        sel = [int(radar.sweep_start_ray_index['data'][0]), int(radar.sweep_end_ray_index['data'][0])]
        values = np.ma.getdata(radar.fields[field]['data'][sel[0] : sel[1]+1, :])
        mask_values = np.ma.getdata(radar.fields[mask_field]['data'][sel[0] : sel[1]+1, :])
        values[np.where(mask_values <= mask_field_thresh)] = 'nan'
        re = r_e[sel[0]: sel[1]+2]
        ze = z_e[sel[0]: sel[1]+2]

        if field in cbar_range:
            vmin, vmax = cbar_range[field]
        else:
            vmin, vmax = np.nanmin(values), np.nanmax(values)
        fig = plt.figure(figsize=(12, 8), dpi=180)
        ax1 = fig.add_subplot(111)
        cs = ax1.pcolormesh(re/1000, ze/1000, values, vmin = vmin, vmax = vmax, cmap = pcm.LangRainbow12)
        llma = plt.scatter(R_cls/1000,  Z_cls/1000, c = abs(Y_min), 
                           edgecolor = 'black', cmap = 'Reds_r', marker = "X", 
                           s = 200, alpha = 0.5, vmin = 0, vmax = distance_max)
        plt.xlim(10,40)
        plt.ylim(0,12)

        # -- cbar
        cb_ax = fig.add_axes([1.01, 0.13, 0.02, 0.75])
        cbar = fig.colorbar(cs, cax=cb_ax)
        cbar.set_label(units[field], size=15)
        cbar.ax.tick_params(labelsize=11)

        llma_ax = fig.add_axes([0.92, 0.13, 0.02, 0.75])
        cbar2 = fig.colorbar(llma, cax=llma_ax)
        cbar2.set_label('Orthogonal Distance (m)', size=15)
        cbar2.ax.tick_params(labelsize=11)

        fig.text(0.55, 0.9,self.subsetter.radar_time.strftime("%d %B %Y %H:%M:%S") + 
                 f' - VHF sources within {ds} m from the RHI scan', 
                 va='center', ha='center', fontsize = 15)
        fig.text(0.5, 0.05, 'Distance from radar (km)', va='center', ha='center', fontsize = 15)
        fig.text(0.09, 0.5, 'Distance above radar (km)', va='center', ha='center', rotation='vertical', fontsize = 15)

        return fig

### Lightning

One file has the whole day.

In [5]:
# Read lma file
lightning_file = '/data/Houston/LIGHTNING/6sensor_minimum/LYLOUT_220617_000000_86400_map500m.nc'
lma_data = xr.open_dataset(lightning_file)
# lma_data.event_time.values = pd.to_datetime(lma_data.event_time.values.astype('M8[us]')).to_pydatetime()

### Subsetting and plotting

In [6]:
import os
from pathlib import Path
figures_out = './cases/figures/0617rhi'

Path(figures_out).mkdir(parents=True, exist_ok=True)

from glob import glob
all_radar_files = sorted(glob("houcsapr2cfrS2.a1_proc/houcsapr2cfrS2.a1.*_proc.nc"))

In [7]:
units={'reflectivity':'Reflectivity (dBZ)',
   'differential_reflectivity':'Differential Reflecivity (dB)',
   'copol_correlation_coeff':'Correlation Coeff. (unitless)',
   'normalized_coherent_power':'Normalized Coherent Power (dB)',
   'specific_differential_phase':'Specific Differential Phase (deg/km)',
   'KDP_CSU':'Specific Differential Phase (deg/km)',
   }

# Defaults to min, max
cbar_range={'reflectivity':(0,65),
    'differential_reflectivity':(-1, 5),
    'copol_correlation_coeff':(.90, 1.0),
    'KDP_CSU':(-15, 15),
   }

seconds = 30
ds = 1000

In [8]:
def plot_one_radar_file(figure_path, radar_filename):
    
    radar = pyart.io.read(radar_filename)
    if radar.scan_type=='rhi':
        subsetter = RadarLMASubsetter(radar,lma_data)
        plotter = OverlayPlotter(subsetter, seconds, ds)
        
        outfile_time_base = os.path.join(figure_path, 
                                         subsetter.radar_time.strftime('%Y%m%d_%H%M%S.'))
        
        plan_ax = subsetter.plot_lightning_near_scan_time(30)
        plan_ax.axis((-95.7, -95.2, 29.5, 30.0))
        plan_ax.figure.savefig(outfile_time_base + 'plan.png')
        plt.close(plan_ax.figure)
                               
        fig = plotter.plot('normalized_coherent_power', units=units,
            cbar_range=cbar_range, distance_max=ds)
        outfilename = outfile_time_base + 'normalized_coherent_power' + '.png'
        fig.savefig(outfilename)
        plt.close(fig)
        
        fields_to_plot = ['reflectivity', 'differential_reflectivity',
                     'specific_differential_phase', 'KDP_CSU',
                     'copol_correlation_coeff']
        for field in fields_to_plot:           
            outfilename = outfile_time_base + field + '.png'
            
            fig = plotter.plot(field, units=units,
                               mask_field = 'normalized_coherent_power',
                               mask_field_thresh=0.5,
                               cbar_range=cbar_range, distance_max=ds)
            fig.savefig(outfilename)
            plt.close(fig)
                               
for radar_file in all_radar_files:
    plot_one_radar_file(figures_out, radar_file)

Radar scan type: rhi
Sweep:  0 of <pyart.core.radar.Radar object at 0x1a615a500>
2022-06-17 19:30:17
22 flashes in 60 second interval near scan.
22 flashes in 60 second interval near scan.
Radar scan type: rhi
Sweep:  0 of <pyart.core.radar.Radar object at 0x1a647bbe0>
2022-06-17 19:30:32
21 flashes in 60 second interval near scan.
21 flashes in 60 second interval near scan.
Radar scan type: rhi
Sweep:  0 of <pyart.core.radar.Radar object at 0x1a67f4a00>
2022-06-17 19:30:46
27 flashes in 60 second interval near scan.
27 flashes in 60 second interval near scan.
Radar scan type: rhi
Sweep:  0 of <pyart.core.radar.Radar object at 0x1a6682140>
2022-06-17 19:31:02
33 flashes in 60 second interval near scan.
33 flashes in 60 second interval near scan.
Radar scan type: rhi
Sweep:  0 of <pyart.core.radar.Radar object at 0x1a79497b0>
2022-06-17 19:31:42
37 flashes in 60 second interval near scan.
37 flashes in 60 second interval near scan.
Radar scan type: rhi
Sweep:  0 of <pyart.core.radar.Rad

In [9]:
# Does not provide a way to get the indices of the points inside the scan,
# so you lose the link to the third dimension.
# lma_in_rhi = lma_shp.intersection(scan_poly)
# for geom in lma_in_rhi.geoms:
#     print(geom)

In [20]:
# Example of finding nearby sources and their closest radar gate.
# R_cls, Z_cls, Y_min = subsetter.find_lightning_distances_near_scan_time(30, 1000)
# cls_r_idx, cls_az_idx, cls_elev_idx = subsetter.radar_gate_at_lightning_source(R_cls, Z_cls, Y_min)